## Build agentic applications using Strands Agents open source SDK by AWS

### Set up AWS environment
- If you are running this notebook in SageMaker Notebook instance, you need to give SageMaker permission to access Bedrock.
- If you are running it in your local environment, follow the steps below to set up proper permission to access your AWS resouces, espeically Bedrock:

Strands supports many different model providers, including Claude models, Amazon models, Cohere models and etc. In this workshop, we are going to use cohere model as the LLM.

To use the examples in this guide, you'll need to configure your environment with AWS credentials that have permissions to invoke the LLM in Bedrock. You can set up your credentials in several ways:

    Environment variables: Set AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, and optionally AWS_SESSION_TOKEN
    AWS credentials file: Configure credentials using aws configure CLI command
    IAM roles: If running on AWS services like EC2, ECS, or Lambda, use IAM roles

Make sure your AWS credentials have the necessary permissions to access Amazon Bedrock. You'll need to enable model access in the Amazon Bedrock console following the AWS documentation here: https://docs.aws.amazon.com/bedrock/latest/userguide/model-access-modify.html

### Install Strands Agents and other packages
AWS Strands Agents Reference: https://strandsagents.com/0.1.x/user-guide/quickstart/

In [1]:
!pip install strands-agents
!pip install strands-agents-tools 
!pip install strands-agents-builder
!pip install pinecone cohere
!pip install -U duckduckgo_search
!pip install pinecone

  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached pydantic_settings-2.9.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached sse_starlette-2.3.6-py3-none-any.whl.metadata (10 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached pydantic_settings-2.9.1-py3-none-any.whl (44 kB)
Using cached python_multipart-0.0.20-py3-none-any.whl (24 kB)
Using cached sse_starlette-2.3.6-py3-none-any.whl (10 kB)
  Using cached aws_requests_auth-0.4.3-py2.py3-none-any.whl.metadata (567 bytes)
  Using cached slack_bolt-1.23.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached slack_sdk-3.35.0-py2.py3-none-any.whl.metadata (15 kB)
Using cached aws_requests_auth-0.4.3-py2.py3-none-any.whl (6.8 kB)
Using cached slack_bolt-1.23.0-py2.py3-none-any.whl (229 kB)
Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
Using cached

In [2]:
from strands import Agent
from strands.models import BedrockModel
from duckduckgo_search import DDGS
from botocore.config import Config
import boto3
import botocore
from datetime import datetime, timedelta, date
import json
from pinecone import Pinecone

### 1. Test some built-in tool

In [3]:
# Simple calculator tool

from strands_tools import calculator

In [17]:
# set the LLM to be Cohere model from Bedrock

bedrock_model = BedrockModel(
  model_id="cohere.command-r-plus-v1:0",
  region_name = "us-west-2",
  temperature=0.3,
)

In [18]:
agent = Agent(tools=[calculator], model=bedrock_model)
agent("What is the square root of 1764")

I will use the calculator tool to find the square root of 1764.
Tool #1: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ sqrt(1764)          │                                                                            │
│  │ Result    │ 42                  │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The square root of 1764 is **42**.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'The square root of 1764 is **42**.'}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'calculator': ToolMetrics(tool={'toolUseId': 'tooluse_-F3cTvhwSCCGX1CCILxQeg', 'name': 'calculator', 'input': {'expression': 'sqrt(1764)', 'mode': 'evaluate', 'precision': 10}}, call_count=1, success_count=1, error_count=0, total_time=0.005624055862426758)}, cycle_durations=[2.1866273880004883], traces=[<strands.telemetry.metrics.Trace object at 0x7fe756114b20>, <strands.telemetry.metrics.Trace object at 0x7fe7561142b0>], accumulated_usage={'inputTokens': 53, 'outputTokens': 59, 'totalTokens': 112}, accumulated_metrics={'latencyMs': 4608}), state={})

### 2. Build custom tool using Python functions

In [6]:
from strands import tool

#### Tool 1: Web search engine

In [7]:
# Tool 1: Web search engine

@tool
def web_search_engine(query: str) -> dict:
    '''
    Searches the internet and retrieves content relevant to the input query
    '''
    # search_results = DDGS().text("Company where Andy Jassy is CEO", max_results=5)
    search_results = DDGS().text(query, max_results=5)
    
    return search_results

#### Tool 2: Word count

In [8]:
@tool
def word_count(text: str) -> int:
    """Count words in text.

    This docstring is used by the LLM to understand the tool's purpose.
    """
    return len(text.split())



#### Tool 3: Pinecone search

In [11]:
pinecone_api_key = '' #Provide Pinecone API key

index_name = 'pinecone-cohere-workshop' #Provide Pinecone Index name that we created in the 'Create Pinecone Index' section
namespace = 'agentic-rag'


pc = Pinecone(api_key=pinecone_api_key)

index = pc.Index(index_name)
print(index.describe_index_stats())

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'agentic-rag': {'vector_count': 880}},
 'total_vector_count': 880,
 'vector_type': 'dense'}


In [12]:
# Initialize Bedrock
config = Config(connect_timeout=5, read_timeout=60, retries={"total_max_attempts": 20, "mode": "adaptive"})
region = 'us-west-2'

bedrock = boto3.client(
                service_name='bedrock-runtime',
                region_name=region,
                endpoint_url=f'https://bedrock-runtime.{region}.amazonaws.com',
                                    config=config)

In [13]:
# Cohere Embedding model in Bedrock

def cohere_embed_query(docs: str, input_type: str) -> list[float]:
    """
    Generate text embedding by using the Cohere Embed model.
    Args:
        docs: string of text to embed.
        input_type: select between [search_document, search_query, classification, clustering, image]
    Returns:
        dict: embeddings in float type.
    """

    body = json.dumps({
        "texts": [docs],
        "input_type": input_type,
        "embedding_types": ["float"]

    })
    
    model_id = 'cohere.embed-english-v3'
    accept = 'application/json' 
    content_type = 'application/json'
    
    # Invoke model 
    response = bedrock.invoke_model(
        body=body, 
        modelId=model_id, 
        accept=accept, 
        contentType=content_type
    )
    
    # Parse response
    response_body = json.loads(response['body'].read())
    embedding = response_body.get('embeddings')['float'][0]
    
    return embedding

In [14]:
# Tool 3: Query Pinecone DB

@tool
def query_pinecone_db(query: str) -> list[dict]:
    '''
    Searches Pinecone vector database index to retrieve documents relevant to the input query. The index contain financial statement data related to Company Compaq
    '''
    
    query_embedding = cohere_embed_query(query, input_type="search_query")

    results = index.query(
        namespace=namespace,
        vector=query_embedding,
        top_k=20,
        include_metadata=True,
    )
    query_results = [{"chunk_text": rec['metadata']['chunk_text']} for rec in results['matches']]
    # Sample results would look like below
    #search_results = [
    #    {"chunk_text": "Amazon revenue for 2025 is $3 Billion"},
    #    {"chunk_text": "Amazon revenue for 2024 is $2 Billion"},
    #    {"chunk_text": "Amazon revenue for 2024 is $1 Billion"},
    #]
    return query_results


In [28]:
SYSTEM_MESSAGE = """## Task and Context
You are an assistant who helps developers use Cohere. 
You are equipped with a number of tools that can provide different types of information. 
If you can't find the information you need from one tool, you should try other tools if there is a possibility that they could provide the information you need.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling

## Tool Guide
For queries related to financial information use pinecone vector database. 
For other queries use websearch

"""

In [29]:
agent = Agent(system_prompt = (SYSTEM_MESSAGE), tools=[web_search_engine,query_pinecone_db, word_count], model=bedrock_model)
response = agent("what's the revenue of the company Compaq in 2001?")

I will search for Compaq's revenue in 2001 in the Pinecone database.
Tool #1: query_pinecone_db
Compaq reported a 2001 consolidated revenue of $33.6 billion.

In [31]:
response = agent("Which company did Rod Canion cofound?")

I will search for which company Rod Canion co-founded.
Tool #3: web_search_engine
Rod Canion co-founded Compaq Computer Corp. in 1982. He also founded Insource Technology Group in 1992.

In [33]:
response = agent("What is 2001 revenue of the company Rod Canion cofound?")

I will search for the company Rod Canion co-founded and then search for its revenue in 2001.
Tool #5: web_search_engine
I found that Rod Canion co-founded Compaq Computer Corp. in 1982. Now I will search for Compaq's revenue in 2001.
Tool #6: query_pinecone_db
Compaq, the company co-founded by Rod Canion, reported a 2001 consolidated revenue of $33.6 billion.